# **Movie Production Business Analysis - IMDB ETL**

- Yvon Bilodeau
- May 2022

## **Business Problem**

For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.


## **The Data**

IMDB Provides Several Files with varied information for Movies, TV Shows, Made for TV Movies, etc.

Overview/Data Dictionary: [https://www.imdb.com/interfaces/](https://www.imdb.com/interfaces/)

Downloads page: [https://datasets.imdbws.com/](https://datasets.imdbws.com/)
        
From their previous research, they realized they want to focus on the following files:

- title.basics.tsv.gz
- title.ratings.tsv.gz
- title.akas.tsv.gz

### **Specifications**

Your stakeholder only wants you to include information for movies based on the following specifications:

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States

### **Download**

In [1]:
# Create url variables
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url ="https://datasets.imdbws.com/title.ratings.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

### **Preprocessing**

In [2]:
import numpy as np
import pandas as pd
import os

#### **Title Basics**

In [ ]:
# Create dataframe
basics_df = pd.read_csv(basics_url,sep='\t', low_memory=False)

- Replace "\N" with np.nan
- Eliminate movies that are null for runtimeMinutes
- Eliminate movies that are null for genre
- Keep only titleType==Movie
- Keep startYear 2000-2022
- Eliminate movies that include  "Documentary" in genre

In [ ]:
basics_df.head()

In [ ]:
# Replace "\N" with np.nan
basics_df.replace({'\\N':np.nan},inplace=True)

In [ ]:
basics_df.head()

In [ ]:
# Eliminate movies that are null for runtimeMinutes
basics_df = basics_df.dropna(subset = ['runtimeMinutes', 'genres', 'startYear'])

In [ ]:
# Keep only titleType == Movie
basics_df = basics_df.loc[basics_df['titleType'] == 'movie']

In [ ]:
basics_df.head(20)

In [ ]:
# Keep startYear 2000-2022
basics_df['startYear'] = basics_df['startYear'].astype(int)
basics_df = basics_df.loc[(basics_df['startYear'] >= 2000) & (basics_df['startYear'] <=2021)]

In [ ]:
basics_df.head()

In [ ]:
# Eliminate movies that include "Documentary" in genre
documentary_filter = basics_df['genres'].str.contains('documentary', case=False)
basics_df = basics_df[~documentary_filter]

In [ ]:
basics_df.head(20)

In [ ]:
# Make directory
os.makedirs('Data/',exist_ok=True) 

In [ ]:
# Confirm directory created
os.listdir("Data/")

In [ ]:
## Save dataframe to file
basics_df.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)

In [3]:
# Open saved file and preview again
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [ ]:
del basics_df

#### **Title Ratings**

- Keep only US entries.
- Replace "\N" with np.nan

In [ ]:
# Create dataframe
ratings_df = pd.read_csv(ratings_url,sep='\t', low_memory=False)

In [ ]:
ratings_df.head()

In [ ]:
# Replace "\N" with np.nan
ratings_df.replace({'\\N':np.nan},inplace=True)

In [ ]:
ratings_df.info()

In [ ]:
## Save dataframe to file
ratings_df.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

In [ ]:
# Open saved file and preview again
ratings_df = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings_df.head()

In [ ]:
del ratings_df

#### **Title Akas**

- Replace "\N" with np.nan (if any)

In [ ]:
# Create dataframe # chunksize creates an iterator
dl_file = "C:/Users/DELL/Downloads/title.akas.tsv.gz"
#akas_df = pd.read_csv(dl_file,sep='\t', low_memory=True, chunksize=100000)
akas_df = pd.read_csv(dl_file,sep='\t', low_memory=True)

In [ ]:
akas_df.head()

In [ ]:
# Replace "\N" with np.nan
akas_df.replace({'\\N':np.nan},inplace=True)

In [ ]:
akas_df.info()

In [ ]:
akas_df = akas_df.loc[akas_df['region'] =='US']

In [ ]:
## Save dataframe to file
akas_df.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)

In [4]:
# Open saved file and preview again
akas_df = pd.read_csv("Data/title_akas.csv.gz", low_memory=True)
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [5]:
# Filter the basics table down to only include the US by using the filter akas dataframe
us_filter = basics_df['tconst'].isin(akas_df['titleId'])
us_filter

0          True
1          True
2          True
3         False
4          True
          ...  
135576     True
135577     True
135578    False
135579     True
135580    False
Name: tconst, Length: 135581, dtype: bool

In [6]:
basics_df = basics_df[us_filter]
basics_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
7,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"
...,...,...,...,...,...,...,...,...,...
135574,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
135575,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
135576,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
135577,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,90,"Action,Adventure,Thriller"


In [7]:
## Save dataframe to file
basics_df.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)